In [21]:
import tensorflow as tf
import keras
import os
import sys
import numpy as np
import segmentation_models as sm
import csv
import medpy.metric.binary as med
# custom loss
import custom_loss as custom
import loss_functions as lossF

In [56]:
os.environ["CUDA_VISIBLE_DEVICES"]='2'

DATA_DIR = '/home/intern/data/data_liverbound_noclip/'
WEIGHT_DIR = './Train_Val_Test_data/saved_weights/'
LIVER_DIR = './Train_Val_Test_data/evaluation/eval_liver/'
RES_DIR = './Train_Val_Test_data/evaluation/'

# Input Data
livers = [LIVER_DIR+x for x in os.listdir(LIVER_DIR)]
livers.sort()

In [57]:
def get_model(img_shape=[None, None]):
    sm.set_framework('keras')
    
    model_seg = sm.Unet(input_shape=img_shape + [5],
                       classes = 1,
                       activation='sigmoid',
                       encoder_weights=None,
                       backbone_name='resnet34',
                       decoder_block_type='transpose',
                       )
    
    inp_all = keras.layers.Input(shape=img_shape + [5], name='input_seg')

    predictions = model_seg(inp_all)
    model = keras.Model(inputs=[inp_all], outputs=predictions)
    
    return model, model_seg

def load_model(model_name, model_loss):
    model, _ = get_model()
    LR = 1e-3
    model.compile(optimizer = keras.optimizers.Adam(LR),
                     loss = model_loss,
                     metrics=[sm.metrics.FScore(per_image=False, name='Dice')])
    model.load_weights(WEIGHT_DIR + model_name)
    
    return model

def get_metric(pr, gt, num):
    
    dice = med.dc(pr,gt)
    precision = med.precision(pr, gt)
    recall = med.recall(pr, gt)
    return [num, dice, precision, recall]

def get_prediction(loss_name, loss):
    model = load_model(loss_name+'_lesion.ckpt', loss)
    
    f = open(RES_DIR+'/metrics/'+loss_name+'_lesion'+'.csv', 'w')
    wr = csv.writer(f)
    wr.writerow([' ','Dice', 'Precision', 'Recall'])
    
    for liver_clip in livers:
        liver = np.load(liver_clip)
        result = np.zeros((512,512,liver.shape[-1]-4))
        for slide in range(liver.shape[-1]-4):
            new_slide = model.predict(liver[np.newaxis,:,:,slide:slide+5])
            new_slide = (new_slide > 0.6).astype(float)
            result[...,slide] = new_slide[0,:,:,0]

        gt = np.load(os.path.join(DATA_DIR, 'segmentation-{}.npy'.format(liver_clip[53:-4])))
        gt = keras.utils.to_categorical(gt, 3)
        gt = gt[...,2]
        gt = gt[...,4:-4]
        metrics = get_metric(result, gt, liver_clip[53:-4])
        wr.writerow(metrics)
        
        print(loss_name+liver_clip[52:], 'finished')
    
    f.close()

In [61]:
get_prediction('Tverfocal03', custom.TverskyFocalLoss(alpha=0.3))


Tverfocal03-0.npy finished
Tverfocal03-1.npy finished
Tverfocal03-10.npy finished
Tverfocal03-100.npy finished
Tverfocal03-101.npy finished
Tverfocal03-102.npy finished
Tverfocal03-103.npy finished
Tverfocal03-104.npy finished
Tverfocal03-105.npy finished
Tverfocal03-106.npy finished
Tverfocal03-107.npy finished
Tverfocal03-108.npy finished
Tverfocal03-109.npy finished
Tverfocal03-11.npy finished
Tverfocal03-110.npy finished
Tverfocal03-111.npy finished
Tverfocal03-112.npy finished
Tverfocal03-113.npy finished
Tverfocal03-114.npy finished
Tverfocal03-115.npy finished
Tverfocal03-116.npy finished
Tverfocal03-117.npy finished
Tverfocal03-118.npy finished
Tverfocal03-119.npy finished
Tverfocal03-12.npy finished
Tverfocal03-120.npy finished
Tverfocal03-121.npy finished
Tverfocal03-122.npy finished
Tverfocal03-123.npy finished
Tverfocal03-124.npy finished
Tverfocal03-125.npy finished
Tverfocal03-126.npy finished
Tverfocal03-127.npy finished
Tverfocal03-128.npy finished
Tverfocal03-129.npy f